In [ ]:
import circuitsvis as cv
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from taker import Model
from taker.hooks import HookConfig
import csv
import json
from datetime import datetime
from os import listdir
from os.path import exists

In [ ]:
with open('../promptsV1.csv', newline='') as f:
    reader = csv.reader(f)
    readdata = list(reader)
    readdata = readdata[:20]

In [ ]:
import sys, os
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [ ]:
max_new_tokens = 200
temperature = 0.3

In [ ]:
orig_df = pd.read_json(f"../gemma9b_results/latest_orig_generation_new.jsonl", lines=True)
def split_at_double_newline(text):
    # Ensure we are only working with strings longer than 15 characters
    if len(text) > 15:
        # Search for the first double newline after the 15th character
        pos = text.find('\n\n', 15)
        if pos != -1:  # Check if double newline was found
            return text[:pos+2], text[pos:]  # Split and remove the newline from the second part
    return text, text  # If no split is required, return the original text and None

orig_df['paragraph1'], orig_df['paragraph2'] = zip(*orig_df['output'].apply(split_at_double_newline))
orig_df['paragraph1'] = orig_df['prompt'].astype(str) + orig_df['paragraph1'].astype(str)
print(repr(orig_df['paragraph1'][0]))

In [ ]:
# idlist = get_ids(orig_df['paragraph1'][0]).squeeze().tolist()
# tokens = tokenizer.convert_ids_to_tokens(idlist)
# print(tokens)
def get_last_segment(text):
    segments = text.split('\n\n')
    return segments[-1].strip() if segments else ''

generated_outputs = []
for i, prompt in enumerate(orig_df['paragraph1'][:1]):
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].squeeze().tolist())
    input_length = len(tokens)
    custom_attention_mask = torch.zeros(input_length,input_length)
    custom_attention_mask[0:tokens.index('\n\n'),0:tokens.index('\n\n')] = 1
    custom_attention_mask[len(tokens)-1,len(tokens)-1] = 1
    inputs['attention_mask'] = custom_attention_mask.unsqueeze(0)
    print(inputs['attention_mask'].shape)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = m.generate(**inputs, max_new_tokens=150, temperature=temperature, do_sample=True)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_text)
    generated_outputs.append(get_last_segment(generated_text))

generated_df = pd.DataFrame({'generated_output': generated_outputs})
print(generated_df)